In [66]:
# - 데이터 가져 오기

import keras
import pandas as pd
df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
print(df.head())
df.shape



                                                   0  1
0  So there is no way for me to plug it in here i...  0
1                        Good case, Excellent value.  1
2                             Great for the jawbone.  1
3  Tied to charger for conversations lasting more...  0
4                                  The mic is great.  1


(1000, 2)

In [67]:
# 일단은 단어 단위로 토큰화를 해준다. 
from keras.preprocessing.text import Tokenizer

tok = Tokenizer()

# 이제 df[0] 즉 0번열 실제로 토큰화 및 텍스트를 단어 번호의 리스트로 변환

tok.fit_on_texts(df[0])
seq = tok.texts_to_sequences(df[0])


In [72]:
# 아마존 리뷰 데이터 활용해서 문장 생성 모델 실습
# 아마존 리뷰 데이터에서 첫번째 글을 보면 문장의 시작에는 'So'가 나오고, '
# So' 다음에는 'there'이 나오며, 'So there' 다음에는 'is'가 나온다. 이러한 관계를 순환신경망에 학습시키면 된다.

df.iloc[0, 0]

'So there is no way for me to plug it in here in the US unless I go by a converter.'

In [73]:
# 텍스트의 시작과 끝을 나타내는 단어를 사전에 추가

tok.word_index['<START>'] = start = len(tok.word_index) + 1
tok.index_word[start] = '<START>'

tok.word_index['<END>'] = end = len(tok.word_index) + 1
tok.index_word[end] = '<END>'


In [74]:
#모든 텍스트의 앞과 뒤에 시작과 끝 표시를 붙여 prev_seq를 만들고, 끝 표시만 붙은 next_seq를 만든다. 
#이렇게 하면 prev_seq와 next_seq는 한 단어씩 어긋나게 된다. 순환신경망에 prev_seq를 입력으로, next_seq를 출력으로 넣어줄 것이다.
prev_seq = []
next_seq = []
for s in seq:
    prev_seq.append([start] + s + [end])
    next_seq.append(s + [end])


In [75]:
# 패딩을 위해 텍스트의 최대 길이를 구한다.
MAXLEN = max(len(s) for s in prev_seq)


# 패딩을 하는데 이전과 달리 뒤에 0을 넣어 채워준다.
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

prev_pad = pad_sequences(prev_seq, MAXLEN, padding='post')
next_pad = pad_sequences(next_seq, MAXLEN, padding='post')

print(prev_pad[0])

print(next_pad[0])

# padding: 문자열, 'pre' 혹은 'post': 각 시퀀스의 처음 혹은 끝을 패딩합니다.


[1881   33  117    5   53  214   11   47    8  155    4   19  337   19
    1  546  416    2  241  190    6  812 1881    0    0    0    0    0
    0    0    0    0]
[  33  117    5   53  214   11   47    8  155    4   19  337   19    1
  546  416    2  241  190    6  812 1881    0    0    0    0    0    0
    0    0    0    0]


In [76]:
# 데이터 분할
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(prev_pad, next_pad, test_size=.2, random_state=1234)

In [77]:
# 모형 만들기
NUM_WORDS = len(tok.index_word) + 1

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, TimeDistributed

rnn = Sequential()
rnn.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))

# return_sequences=True로 모든 입력에 대해 출력을 내놓게 한다.
rnn.add(LSTM(16, return_sequences=True))

# 엑티베이션 함수도 시그모이드가 아닌 여러 개 확률 출력 softmax사용
rnn.add(TimeDistributed(Dense(NUM_WORDS, activation='softmax')))

# 모델 구성 확인
rnn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 32, 8)             15056     
                                                                 
 lstm_3 (LSTM)               (None, 32, 16)            1600      
                                                                 
 time_distributed_2 (TimeDis  (None, 32, 1882)         31994     
 tributed)                                                       
                                                                 
Total params: 48,650
Trainable params: 48,650
Non-trainable params: 0
_________________________________________________________________


In [78]:
import numpy

# 출력의 형태 맞추기
y_train.shape

(800, 32)

In [79]:

# 출력의 형태가 (None, 32, 1881)인데 데이터는 (800, 32)의 형태이므로 차원이 맞지 않는다. 끝에 1차원을 덧붙여서 형태를 맞춰준다.
y_train_dims = numpy.expand_dims(y_train, 2)
y_train_dims.shape


(800, 32, 1)

In [80]:
# 학습 하기
from keras.optimizers import Adam

rnn.compile(optimizer=Adam(lr=.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')


C:\Users\user\AppData\Local\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [81]:

rnn.fit(x_train, y_train_dims, epochs=30)

y_train.shape


Epoch 1/30
25/25 [==============================] - 3s 37ms/step - loss: 6.1842 - accuracy: 0.0719
Epoch 2/30
25/25 [==============================] - 1s 40ms/step - loss: 5.2601 - accuracy: 0.1602
Epoch 3/30
25/25 [==============================] - 1s 55ms/step - loss: 4.9053 - accuracy: 0.1892
Epoch 4/30
25/25 [==============================] - 1s 37ms/step - loss: 4.6020 - accuracy: 0.2120
Epoch 5/30
25/25 [==============================] - 1s 38ms/step - loss: 4.3636 - accuracy: 0.2273
Epoch 6/30
25/25 [==============================] - 1s 39ms/step - loss: 4.1775 - accuracy: 0.2399
Epoch 7/30
25/25 [==============================] - 1s 37ms/step - loss: 4.0068 - accuracy: 0.2560
Epoch 8/30
25/25 [==============================] - 1s 37ms/step - loss: 3.8711 - accuracy: 0.2645
Epoch 9/30
25/25 [==============================] - 1s 38ms/step - loss: 3.7469 - accuracy: 0.2711
Epoch 10/30
25/25 [==============================] - 1s 36ms/step - loss: 3.6550 - accuracy: 0.2849
Epoch 11/

(800, 32)

In [82]:
# 언어 모형을 이용해 문장의 다음 단어를 예측해보자. 예를 들어 첫번째 리뷰의 앞 10단어는 다음과 같다.
[tok.index_word[i] for i in prev_seq[2][:10]]


['<END>', 'great', 'for', 'the', 'jawbone', '<END>']

In [83]:
# 이어서 나올 단어는 'it'이다.

i = prev_seq[0][10]
tok.index_word[i]


'it'

In [84]:
# 이제 RNN으로 예측을 해보자.
new_sentence = [prev_seq[0][:10]]

# 패딩을 하고
new_pad = pad_sequences(new_sentence, MAXLEN, padding='post')

[tok.index_word[i] for i in prev_seq[0][:10]]


['<END>', 'so', 'there', 'is', 'no', 'way', 'for', 'me', 'to', 'plug']

In [85]:
# 예측을 한다.
next_words = rnn.predict(new_pad)


# 1개의 텍스트에 대해 32단어 길이로 1881종의 단어에 대한 예측이 나왔다.
next_words.shape


1/1 [==============================] - 1s 834ms/step


(1, 32, 1882)

In [86]:
# 가장 확률이 높은 단어는 4번이다.
next_words[0, 10].argmax()

4

In [87]:
tok.index_word[4] 

'it'

In [88]:
# 이를 처음부터 끝까지 반복하게 해서 완전 문장 생성.
new_sentence = [[start]]
new_pad = pad_sequences(new_sentence, MAXLEN, padding='post')

for i in range(MAXLEN - 1):
    next_words = rnn.predict(new_pad) # 예측
    word = next_words[0, i].argmax()  # 가장 확률이 높은 단어 선정
    print(tok.index_word[word])       # 단어 출력
    
    new_pad[0, i + 1] = word          # 선정 단어를 추가
    if word == end:                   # 문장이 끝나면 중단
        break


1/1 [==============================] - 0s 17ms/step
i
1/1 [==============================] - 0s 16ms/step
have
1/1 [==============================] - 0s 16ms/step
had
1/1 [==============================] - 0s 16ms/step
this
1/1 [==============================] - 0s 18ms/step
phone
1/1 [==============================] - 0s 16ms/step
<END>


In [89]:
# 위의 방식으로 하면 매번 같은 문장이 만들어지므로 다양성이 부족하다. 확률이 가장 높은 단어를 선택하는 대신, 
# 단어를 확률에 따라 무작위로 추출하게 하자.

import numpy.random
new_sentence = [[start]]
new_pad = pad_sequences(new_sentence, MAXLEN, padding='post')

for i in range(MAXLEN - 1):
    next_words = rnn.predict(new_pad)

    # 확률에 따라 단어를 무작위로 추출
    word = numpy.random.choice(NUM_WORDS, p=next_words[0, i])

    print(tok.index_word[word])
    new_pad[0, i + 1] = word
    if word == end:
        break


1/1 [==============================] - 0s 17ms/step
great
1/1 [==============================] - 0s 16ms/step
price
1/1 [==============================] - 0s 17ms/step
<END>
